In [3]:
#Kathrin Fischer
import yt
import matplotlib.pyplot as plt
from astropy import units as u, constants  as c
import numpy as np
from unyt import cm, s, erg, K, sr
pc = c.pc.cgs.value
kB  = c.k_B.cgs.value
Msun = c.M_sun.cgs.value
G = c.G.cgs.value
Myr = u.Myr.in_units("s")
eV = 1.60218e-12   # eV in erg

In [2]:
#versch vmin vmax
def _j_halpha_R(field, data):
    T = data[("gas", "temperature")].v
    T4 = T / 1e4
    number_density = data[("gas", "number_density")].v
    ion_fraction = data[("flash", "ihp ")].v
    ne_np = (number_density * ion_fraction) ** 2
    return 2.82e-26 * (T4 ** (-0.942 - 0.03 * np.log(T4))) * ne_np * erg / cm**3 / s / sr

def _j_halpha_C(field, data):
    T = data[("gas", "temperature")].v
    number_density = data[("gas", "number_density")].v
    ion_fraction = data[("flash", "ihp ")].v
    Ha_fraction = data[("flash", "iha ")]    
    ne = number_density * ion_fraction
    nH = number_density * Ha_fraction

    def lambda_13(T):
        if 4000 <= T <= 25000:
            return 0.35 - 2.62e-7*T - 8.15e-11*T**2 + 6.19e-15*T**3
        elif 25000 < T <= 500000:
            return 0.276 + 4.99e-6*T - 8.85e-12*T**2 + 7.18e-18*T**3
        elif T < 4000:
            T = 4000
            return 0.35 - 2.62e-7*T - 8.15e-11*T**2 + 6.19e-15*T**3
        elif T > 500000: 
            T = 500000
            return 0.276 + 4.99e-6*T - 8.85e-12*T**2 + 7.18e-18*T**3
    
    lambda_13_vec = np.vectorize(lambda_13)
    return (1.30e-17 / (4 * np.pi)) * (lambda_13_vec(T) / np.sqrt(T)) * np.exp(-12.1 * 11604.5 / T) * ne * nH * erg / cm**3 / s / sr

def _j_halpha_RC(field, data):
    j_halpha_R = data[("gas", "j_halpha_R")]
    j_halpha_C = data[("gas", "j_halpha_C")]
    #return (j_halpha_R + j_halpha_C) * 4 * np.pi
    return (j_halpha_R + j_halpha_C) 

ctr = [-80*pc, -150*pc, 0]
files = ["SILCC_hdf5_plt_cnt_1080", "SILCC_hdf5_plt_cnt_1140"]
names = ["10.8Myr", "11.4Myr"]
directions = ["z"]
#directions = ["z", "y"]
#emission_types = ["j_halpha_RC"]
emission_types = ["j_halpha_RC", "j_halpha_R", "j_halpha_C"]
emission_limits_proj = {"j_halpha_RC": (1e-9, 1e-4), # davor 1e-9 - 1e-2
    "j_halpha_C": (1e-9, 1e-4), #1e-13 - 1e-3
    "j_halpha_R": (1e-9, 1e-4),} #1e-9 - 1e-5
emission_limits_slc = {"j_halpha_RC": (1e-34, 1e-24), #1e-32 - 1e-24
    "j_halpha_C": (1e-34, 1e-24), #1e-40 - 1e-26
    "j_halpha_R": (1e-34, 1e-24),} #1e-34 - 1e-25

for f, name in zip(files, names):
    print(f)
    ds = yt.load("data/"+f)
    yt.add_field(("gas", "j_halpha_R"), function=_j_halpha_R, units="erg/cm**3/s/sr", sampling_type="cell", force_override=True)
    yt.add_field(("gas", "j_halpha_C"), function=_j_halpha_C, units="erg/cm**3/s/sr", sampling_type="cell", force_override=True)
    yt.add_field(("gas", "j_halpha_RC"), function=_j_halpha_RC, units="erg/cm**3/s/sr", sampling_type="cell", force_override=True)
    
    for direction in directions:
        for emission in emission_types:
            slc = yt.SlicePlot(ds, direction, [("gas", emission)], center=ctr)
            vmin_s, vmax_s = emission_limits_slc[emission]
            slc.set_zlim(("gas", emission), vmin_s, vmax_s)
            slc.save(f"{direction}_{emission}_slice_plot_{name}.pdf")
            
            proj = yt.ProjectionPlot(ds, direction, [("gas", emission)], center=ctr)
            vmin_p, vmax_p = emission_limits_proj[emission]
            proj.set_zlim(("gas", emission), vmin_p, vmax_p)
            proj.save(f"{direction}_{emission}_proj_plot_{name}.pdf")


SILCC_hdf5_plt_cnt_1080


yt : [INFO     ] 2025-03-17 10:52:40,624 Parameters: current_time              = 340597785179995.1
yt : [INFO     ] 2025-03-17 10:52:40,626 Parameters: domain_dimensions         = [8 8 8]
yt : [INFO     ] 2025-03-17 10:52:40,629 Parameters: domain_left_edge          = [-7.715e+20 -7.715e+20 -7.715e+20]
yt : [INFO     ] 2025-03-17 10:52:40,630 Parameters: domain_right_edge         = [7.715e+20 7.715e+20 7.715e+20]
yt : [INFO     ] 2025-03-17 10:52:40,635 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2025-03-17 10:53:00,814 xlim = -1018354206519309369344.000000 524645793480690630656.000000
yt : [INFO     ] 2025-03-17 10:53:00,816 ylim = -1234351637223705149440.000000 308648362776294916096.000000
yt : [INFO     ] 2025-03-17 10:53:00,824 xlim = -1018354206519309369344.000000 524645793480690630656.000000
yt : [INFO     ] 2025-03-17 10:53:00,825 ylim = -1234351637223705149440.000000 308648362776294916096.000000
yt : [INFO     ] 2025-03-17 10:53:00,833 Making a fixed resolution b

SILCC_hdf5_plt_cnt_1140


yt : [INFO     ] 2025-03-17 10:55:32,424 xlim = -1018354206519309369344.000000 524645793480690630656.000000
yt : [INFO     ] 2025-03-17 10:55:32,425 ylim = -1234351637223705149440.000000 308648362776294916096.000000
yt : [INFO     ] 2025-03-17 10:55:32,429 xlim = -1018354206519309369344.000000 524645793480690630656.000000
yt : [INFO     ] 2025-03-17 10:55:32,430 ylim = -1234351637223705149440.000000 308648362776294916096.000000
yt : [INFO     ] 2025-03-17 10:55:32,433 Making a fixed resolution buffer of (('gas', 'j_halpha_RC')) 800 by 800
yt : [INFO     ] 2025-03-17 10:55:33,304 Saving plot z_j_halpha_RC_slice_plot_11.4Myr.pdf
yt : [INFO     ] 2025-03-17 10:56:31,287 Projection completed
yt : [INFO     ] 2025-03-17 10:56:31,295 xlim = -1018354206519309369344.000000 524645793480690630656.000000
yt : [INFO     ] 2025-03-17 10:56:31,296 ylim = -1234351637223705149440.000000 308648362776294916096.000000
yt : [INFO     ] 2025-03-17 10:56:31,304 xlim = -1018354206519309369344.000000 52464579

In [4]:
#vmin vmax für alle
def _j_halpha_R(field, data):
    T = data[("gas", "temperature")].v
    T4 = T / 1e4
    number_density = data[("gas", "number_density")].v
    ion_fraction = data[("flash", "ihp ")].v
    ne_np = (number_density * ion_fraction) ** 2
    return 2.82e-26 * (T4 ** (-0.942 - 0.03 * np.log(T4))) * ne_np * erg / cm**3 / s / sr

def _j_halpha_C(field, data):
    T = data[("gas", "temperature")].v
    number_density = data[("gas", "number_density")].v
    ion_fraction = data[("flash", "ihp ")].v
    Ha_fraction = data[("flash", "iha ")]    
    ne = number_density * ion_fraction
    nH = number_density * Ha_fraction

    def lambda_13(T):
        if 4000 <= T <= 25000:
            return 0.35 - 2.62e-7*T - 8.15e-11*T**2 + 6.19e-15*T**3
        elif 25000 < T <= 500000:
            return 0.276 + 4.99e-6*T - 8.85e-12*T**2 + 7.18e-18*T**3
        elif T < 4000:
            T = 4000
            return 0.35 - 2.62e-7*T - 8.15e-11*T**2 + 6.19e-15*T**3
        elif T > 500000: 
            T = 500000
            return 0.276 + 4.99e-6*T - 8.85e-12*T**2 + 7.18e-18*T**3
    
    lambda_13_vec = np.vectorize(lambda_13)
    return (1.30e-17 / (4 * np.pi)) * (lambda_13_vec(T) / np.sqrt(T)) * np.exp(-12.1 * 11604.5 / T) * ne * nH * erg / cm**3 / s / sr

def _j_halpha_RC(field, data):
    j_halpha_R = data[("gas", "j_halpha_R")]
    j_halpha_C = data[("gas", "j_halpha_C")]
    return (j_halpha_R + j_halpha_C) * 4 * np.pi

ctr = [-80*pc, -150*pc, 0]
files = ["SILCC_hdf5_plt_cnt_1080", "SILCC_hdf5_plt_cnt_1140"]
names = ["10.8Myr", "11.4Myr"]
directions = ["z"]
#directions = ["z", "y"]
#emission_types = ["j_halpha_RC"]
emission_types = ["j_halpha_RC", "j_halpha_R", "j_halpha_C"]
vmin = 1e-13
vmax = 1e-2

for f, name in zip(files, names):
    print(f)
    ds = yt.load("data/"+f)
    yt.add_field(("gas", "j_halpha_R"), function=_j_halpha_R, units="erg/cm**3/s/sr", sampling_type="cell", force_override=True)
    yt.add_field(("gas", "j_halpha_C"), function=_j_halpha_C, units="erg/cm**3/s/sr", sampling_type="cell", force_override=True)
    yt.add_field(("gas", "j_halpha_RC"), function=_j_halpha_RC, units="erg/cm**3/s/sr", sampling_type="cell", force_override=True)
    
    for direction in directions:
        for emission in emission_types:
           # slc = yt.SlicePlot(ds, direction, [("gas", emission)], center=ctr)
            #slc.save(f"{direction}_{emission}_slice_plot_{name}.pdf")

            proj = yt.ProjectionPlot(ds, direction, [("gas", emission)], center=ctr)
            proj.set_zlim(("gas", emission), vmin, vmax)
            proj.save(f"{direction}_{emission}_proj_plot_{name}.pdf")


SILCC_hdf5_plt_cnt_1080


yt : [INFO     ] 2025-03-13 16:00:29,568 Parameters: current_time              = 340597785179995.1
yt : [INFO     ] 2025-03-13 16:00:29,571 Parameters: domain_dimensions         = [8 8 8]
yt : [INFO     ] 2025-03-13 16:00:29,579 Parameters: domain_left_edge          = [-7.715e+20 -7.715e+20 -7.715e+20]
yt : [INFO     ] 2025-03-13 16:00:29,582 Parameters: domain_right_edge         = [7.715e+20 7.715e+20 7.715e+20]
yt : [INFO     ] 2025-03-13 16:00:29,587 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2025-03-13 16:01:47,418 Projection completed
yt : [INFO     ] 2025-03-13 16:01:47,426 xlim = -1018354206519309369344.000000 524645793480690630656.000000
yt : [INFO     ] 2025-03-13 16:01:47,427 ylim = -1234351637223705149440.000000 308648362776294916096.000000
yt : [INFO     ] 2025-03-13 16:01:47,438 xlim = -1018354206519309369344.000000 524645793480690630656.000000
yt : [INFO     ] 2025-03-13 16:01:47,440 ylim = -1234351637223705149440.000000 308648362776294916096.000000
yt : [

SILCC_hdf5_plt_cnt_1140


yt : [INFO     ] 2025-03-13 16:02:57,505 Parameters: current_time              = 359537415765615.6
yt : [INFO     ] 2025-03-13 16:02:57,507 Parameters: domain_dimensions         = [8 8 8]
yt : [INFO     ] 2025-03-13 16:02:57,508 Parameters: domain_left_edge          = [-7.715e+20 -7.715e+20 -7.715e+20]
yt : [INFO     ] 2025-03-13 16:02:57,509 Parameters: domain_right_edge         = [7.715e+20 7.715e+20 7.715e+20]
yt : [INFO     ] 2025-03-13 16:02:57,510 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2025-03-13 16:04:11,115 Projection completed
yt : [INFO     ] 2025-03-13 16:04:11,119 xlim = -1018354206519309369344.000000 524645793480690630656.000000
yt : [INFO     ] 2025-03-13 16:04:11,121 ylim = -1234351637223705149440.000000 308648362776294916096.000000
yt : [INFO     ] 2025-03-13 16:04:11,134 xlim = -1018354206519309369344.000000 524645793480690630656.000000
yt : [INFO     ] 2025-03-13 16:04:11,135 ylim = -1234351637223705149440.000000 308648362776294916096.000000
yt : [